ClearML Template

In [1]:
!pip install clearml

from clearml import Task, Dataset

import pandas as pd
import os
import time


os.environ["CLEARML_API_ACCESS_KEY"] = "BF3OQG84RHSYHO86OIYII6Y7VVVMDV"
os.environ["CLEARML_API_SECRET_KEY"] = "7qQDrf96hXOsbmEio_kUbkEjUqqKaMyq4HB4K1fzOWHjKPelw2HEJdB3RrUvh4FZlA8"
os.environ["CLEARML_API_HOST"] = "https://api.clear.ml"
os.environ["CLEARML_WEB_HOST"] = "https://app.clear.ml"
os.environ["CLEARML_FILES_HOST"] = "https://files.clear.ml"

In [2]:
testDataPath = Dataset.get(dataset_id="a5958fdaf8cd449bbadc2bf57e5e797b").get_local_copy()
print(testDataPath)

/root/.clearml/cache/storage_manager/datasets/ds_a5958fdaf8cd449bbadc2bf57e5e797b


In [3]:
test_val_train_folder=os.listdir(testDataPath)

print(testDataPath+"/"+test_val_train_folder[0])
print(testDataPath+"/"+test_val_train_folder[1])
print(testDataPath+"/"+test_val_train_folder[2])

/root/.clearml/cache/storage_manager/datasets/ds_a5958fdaf8cd449bbadc2bf57e5e797b/train_filtered.tsv
/root/.clearml/cache/storage_manager/datasets/ds_a5958fdaf8cd449bbadc2bf57e5e797b/valid_filtered.tsv
/root/.clearml/cache/storage_manager/datasets/ds_a5958fdaf8cd449bbadc2bf57e5e797b/test_filtered.tsv


In [4]:
# wczytujemy do DataFrame
columns = [
    "id", "label", "statement", "subjects", "speaker", "speaker_job", "state_info",
    "party_affiliation", "barely_true", "false", "half_true",
    "mostly_true", "pants_on_fire", "context"
]

trainDF = pd.read_csv(testDataPath+"/"+test_val_train_folder[0], sep='\t', names=columns, header=None)
valDF = pd.read_csv(testDataPath+"/"+test_val_train_folder[1], sep='\t', names=columns, header=None)
testDF = pd.read_csv(testDataPath+"/"+test_val_train_folder[2], sep='\t',names=columns, header=None )

df_train = trainDF
df_valid = valDF
df_test = testDF

In [5]:
testDF.head(1)

,id,label,statement,subjects,speaker,speaker_job,state_info,party_affiliation,barely_true,false,half_true,mostly_true,pants_on_fire,context
0,11972.json,True,Building a wall on the U.S.-Mexico border will take literally years.,immigration,rick-perry,Governor,Texas,republican,30,30,42,23,18,Radio interview


# Pre processing


### Importy

In [6]:
import re

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
import spacy
from sklearn.metrics import accuracy_score, f1_score

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Stop words

In [7]:
stop_words = set(stopwords.words('english'))

In [8]:
#2 wersja z użyciem spacy

# nlp = spacy.load('en_core_web_sm')

#można tez połączyć te dwie metody robiąc listę każdej i potem .concat()

### Czyszczenie tekstu

In [9]:
def clean_text(text):
    text = text.lower()  # lowercase
    text = re.sub(r'\d+', '', text)  # remove all digits from text
    text = re.sub(r'[^\w\s]', '', text)  # remove punctuation
    text = re.sub(r'<[^>]+>', '', text)  # remove html tags
    text = re.sub(r'<[^>]*>', '', text)  # additional html tag removal
    emojis = re.findall(r'(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)  # find emojis
    text = re.sub(r'[\W]+', ' ', text) + ' '.join(emojis).replace('-', '')
    text = text.strip() # remove
    return text

### Lematyzacja

In [10]:
lemmatizer = WordNetLemmatizer()

### Stemming

In [11]:
porter = PorterStemmer()

### Tokenizacja + poprzednie funkcje

In [12]:
def custom_tokenizer(text, cfg):
    text = clean_text(text)
    tokens = word_tokenize(text)

    if cfg["lemmatization"]:
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
    if cfg["stemming"]:
        tokens = [porter.stem(token) for token in tokens]
    if cfg["remove_stopwords"]:
        tokens = [t for t in tokens if t not in stop_words]
    tokens = [t for t in tokens if len(t) >= cfg["min_token_length"]]

    if cfg["concat_to_sentence"]:
        return " ".join(tokens)
    return tokens


# Metoda Embedding

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tf_idf(X_train, X_val, X_test):
  vectorizer = TfidfVectorizer()  # tutaj można dodać parametry
  X_train_tfidf = vectorizer.fit_transform(X_train)

  X_val_tfidf = vectorizer.transform(X_val)
  X_test_tfidf = vectorizer.transform(X_test)
  return vectorizer, X_train_tfidf, X_val_tfidf, X_test_tfidf

# Metoda Klasyfikacji

In [14]:
from sklearn.ensemble import RandomForestClassifier

def random_forest(X_train, y_train):
  model = RandomForestClassifier(n_estimators=100, random_state=42)
  model.fit(X_train, y_train)
  return model

# Wywołanie Taska

In [15]:
task_name = "TF-IDF + Random Forest, full statement"

# Tworzenie taska
task = Task.init(project_name="FakeNewsDetection", task_name=task_name)

# config do preprocesingu
cfg_preprocessing ={
    "tokenizer": True,
    "lemmatization": True,
    "stemming": False,
    "remove_stopwords": True,
    "min_token_length": 3, # wywala slowa któtsze niz 3
    "concat_to_sentence": False # czy łaczymy w zdania (True) czy zostawimy jako wyrazy (False)
}

# preprocessing
df_train["tokens"] = df_train["statement"].apply(lambda x: custom_tokenizer(x, cfg_preprocessing))
df_valid["tokens"] = df_valid["statement"].apply(lambda x: custom_tokenizer(x, cfg_preprocessing))
df_test["tokens"] = df_test["statement"].apply(lambda x: custom_tokenizer(x, cfg_preprocessing))


# wyswietl przyklad preprocesingu
print(df_train[["statement", "tokens"]].head(3))


# metoda
start_time = time.time()

y_train = df_train["label"]
y_val = df_valid["label"]
y_test = df_test["label"]

vectorizer, X_train, X_val, X_test = tf_idf(df_train["statement"], df_valid["statement"], df_test["statement"])

model = random_forest(X_train, y_train)

end_time = time.time()
training_duration = end_time - start_time

# Tutaj kod ..... = test_loss, test_accuracy = model.evaluate(X_test, y_test)

val_preds = model.predict(X_val)
test_preds = model.predict(X_test)

val_acc = accuracy_score(y_val, val_preds)
test_acc = accuracy_score(y_test, test_preds)

logger = task.get_logger()
logger.report_scalar("Duration", "Duration (s)", value=training_duration, iteration=0)
logger.report_scalar("Accuracy", "Validation Accuracy", value=val_acc, iteration=0)
logger.report_scalar("Accuracy", "Test Accuracy", value=test_acc, iteration=0)

# Zamykanie taska
task.close()

ClearML Task: created new task id=bd2b7822708c4c13b0d049dcaa36467f
ClearML results page: https://app.clear.ml/projects/25cc2e9801f0421ba3bf3ef6bcb791c7/experiments/bd2b7822708c4c13b0d049dcaa36467f/output/log
                                                                                                                                                     statement  \
0                                                                           Says the Annies List political group supports third-trimester abortions on demand.   
1                                                                               Health care reform legislation is likely to mandate free sex change surgeries.   
2  The Chicago Bears have had more starting quarterbacks in the last 10 years than the total number of tenured (UW) faculty fired during the last two decades.   

                                                                                                          tokens  
0                           

ValueError: Found input variables with inconsistent numbers of samples: [432, 461]